In [1]:
import pycuda
import pycuda.driver as cuda
from pycuda.compiler import SourceModule
import pycuda.autoinit
import numpy as np
import time

In [2]:
def compute_pi_cpu(n_points):
    x_rand = np.random.rand(n_points)
    y_rand = np.random.rand(n_points)

    n_inside = 0
    for i in range(n_points):
        n_inside += np.sqrt(x_rand[i]**2 + y_rand[i]**2) <= 1.0

    pi = 4 * n_inside/n_points

    return pi

In [19]:
tic = time.time()
print(compute_pi_cpu(1024))
toc = time.time()

print("Time to execute cpu version: {:f} seconds".format(toc-tic))

3.12890625
Time to execute cpu version: 0.010859 seconds


In [72]:
pi_kernel_src = """
//Based on Stroustrup, adapted for CUDA
//pseudorandom numbers
__device__ float generateRandomNumber(long& last_draw) {
    last_draw = last_draw*1103515245 + 12345;
    long abs = last_draw & 0x7fffffff;
    return abs / 2147483648.0; 
}

__global__ void computePi(unsigned int* inside, unsigned int seed) {
    __shared__ unsigned int inside_shared[512];
    
    //1 generate random numbers
    unsigned int tid = threadIdx.x;
    
    long rand_seed = seed + tid;
    float x = generateRandomNumber(rand_seed);
    float y = generateRandomNumber(rand_seed);
    
    //2 compute the radius from the origin
    float r = sqrt(x*x + y*y);
    
    //3 check if inside circle and write to memory
    if (r <= 1) {
            num_inside += 1;
    }
    inside_shared[tid] = num_inside;

    /////////////////////////
    //Shared memory reduction
    /////////////////////////

    // Synchronze so that all thread see the same shared memory
    __syncthreads();

    // Find the sum in shared memory
    //Reduce from 512 to 256 elements
    if (threadIdx.x < 256) {
        inside_shared[threadIdx.x] = inside_shared[threadIdx.x] + inside_shared[threadIdx.x + 256];
    }
    __syncthreads();

    //Reduce from 256 to 128 elements
    if (threadIdx.x < 128) {
        inside_shared[threadIdx.x] = inside_shared[threadIdx.x] + inside_shared[threadIdx.x + 128];
    }
    __syncthreads();

    //Reduce from 128 to 64 elements
    if (threadIdx.x < 64) {
        inside_shared[threadIdx.x] = inside_shared[threadIdx.x] + inside_shared[threadIdx.x + 64];
    }
    __syncthreads();

    //Reduce from 32 to 16 elements
    //Since we here have only one active warp (threadIdx.x > 32)
    //we do not need to call syncthreads anymore
    volatile unsigned int* p = &inside_shared[0]; //To help the compiler not cache this variable...
    if (threadIdx.x < 32) {
        p[threadIdx.x] = p[threadIdx.x] + p[threadIdx.x + 32];
        p[threadIdx.x] = p[threadIdx.x] + p[threadIdx.x + 16];
        p[threadIdx.x] = p[threadIdx.x] + p[threadIdx.x + 8];
        p[threadIdx.x] = p[threadIdx.x] + p[threadIdx.x + 4];
        p[threadIdx.x] = p[threadIdx.x] + p[threadIdx.x + 2];
        p[threadIdx.x] = p[threadIdx.x] + p[threadIdx.x + 1];
    }

    // Finally write out to output
    // NOTE: We have 512 threads, but only thread 0 writes to memory
    if (threadIdx.x == 0) {
        inside[bid] = p[0];
    }
}
"""

mod = SourceModule(pi_kernel_src)
func = mod.get_function("computePi")

In [80]:
def compute_pi_gpu(n_points, threads_per_block=512):
    assert(n_points % threads_per_block == 0)

    #Allocate output data on the GPU
    #Bytes per unsigned int:
    bytes_per_uint = 4
    inside_gpu = cuda.mem_alloc(bytes_per_uint*(n_points//threads_per_block))

    #Execute the pi-kernel
    num_blocks = n_points//threads_per_block
    block=(threads_per_block,1,1)
    grid=(num_blocks,1,1)
    compute_pi_gpu_kernel(inside_gpu, np.uint32(iterations_per_thread), np.uint32(time.time()), block=(threads_per_block,1,1), grid=(num_blocks,1,1))
    
    #Allocate memory to download to on the CPU
    inside_cpu = np.empty((n_points//threads_per_block), dtype=np.uint32)

    #Download from the GPU to the CPU
    cuda.memcpy_dtoh(inside_cpu, inside_gpu)

    #Count number of points inside
    n_inside = np.sum(inside_cpu)

    #We can estimate pi by the following formula:
    pi = 4*n_inside/n_points

    return pi

tic = time.time()
print(compute_pi_gpu(5120000, 10000, 512))
toc = time.time()

print("Time taken: {:f} seconds".format(toc-tic))

In [81]:
tic = time.time()
print(compute_pi_gpu(1024))
toc = time.time()

print("Time to execute gpu version: {:f} seconds".format(toc-tic))

4.0
Time to execute gpu version: 0.001571 seconds
